In [2325]:
import pandas as pd
import numpy as np
import re

In [2326]:
df = pd.read_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/Samsung_new.csv')

In [2327]:
df.duplicated().sum()

51

In [2328]:
df_S= df.drop_duplicates()

In [2329]:
df_S.shape

(306, 5)

In [2330]:
pd.options.mode.chained_assignment = None  # default='warn'

In [2331]:
df_S['Name']=df_S['Company Name'].str.extract('(SAMSUNG)')
df_S['Name']=df_S['Name'].fillna("SAMSUNG")

In [2332]:
df_S['test'] = df_S['Company Name'].str.extract('(\([^)]*\,)')
df_S['test1']=df_S['test'].astype(str)
df_S['Color'] = df_S['test1'].apply(lambda x: x.replace('(','').replace(',','').replace('?','').replace('nan','Not Mentioned'))

In [2333]:
df_S['test2'] = df_S['Company Name'].str.split('(').str[0]
df_S['Model']= df_S['test2'].str.split().str[1:].str.join(' ')
df_S['Model']=df_S['Model'].apply(lambda x : str(x).replace('without charger','').replace('with Snapdragon 888','').replace('- White',''))

In [2334]:
def match(x):
    if re.search("Ratings",x):
        return x
    else:
        return "0"

In [2335]:
df_S['test3']=df_S['Ratings'].apply(match)
df_S['test5']=df_S['test3'].apply(lambda x: x[x.find("SAMSUNG") : x.find("Ratings")])
df_S['Ratings(*)']=df_S['test3'].str.extract('(4\.\d|3\.\d)')
df_S['Ratings(*)']=df_S['Ratings(*)'].fillna(0)

In [2336]:
df_S['test4']=df_S['test3'].apply(lambda x: x[x.find("Ratings") : x.find("Reviews")])
df_S['Reviews']=df_S['test4'].apply(lambda x: x.replace('Ratings','').replace('&','').replace(',',''))
df_S['Reviews']=df_S['Reviews'].str.strip()

In [2337]:
def remove_whitespace(s):
    return ' '.join(s.split())

In [2338]:
df_S['MobilesSold']=df_S['Reviews'].apply(remove_whitespace)
df_S['MobilesSold']=df_S['MobilesSold'].replace('','0')
df_S['MobilesSold']=df_S['MobilesSold'].astype(int)

In [2339]:
df_S['test6']=df_S['Mobile Price'].apply(lambda x: x.replace('%','').replace('offFree','').replace('delivery','').replace('off','').replace('Free delivery','').replace('Free',''))

In [2340]:
df_S['OfferInPerentage'] =df_S['offer'].str.extract('(\d+)')
df_S['OfferInPerentage']=df_S['OfferInPerentage'].fillna(0)
df_S['OfferInPerentage']=df_S['OfferInPerentage'].astype(int)

In [2341]:
df_S['test7']=df_S['test6'].str.extract('(\₹[^)]*\₹)')

In [2342]:
df_S['test7'] = np.where(df_S['test7'].isnull(),df_S['test6'],df_S['test7'])

In [2343]:
df_S['DiscountedPrice']=df_S['test7'].apply(lambda x:str(x).replace('₹','').replace(',','').replace('Price: Not Available',''))
#Replacing NAN Values from Price(with discount) column
df_S['DiscountedPrice']=df_S['DiscountedPrice'].replace('','0')
df_S['DiscountedPrice']=df_S['DiscountedPrice'].astype(int)

In [2344]:
df_S['MaxRetailPrice'] = (df_S.DiscountedPrice + ((df_S.OfferInPerentage/100) * df_S.DiscountedPrice))
df_S['MaxRetailPrice']=df_S['MaxRetailPrice'].round(0)

In [2345]:
df_S['Sales(₹ CR)']=(df_S.DiscountedPrice * df_S.MobilesSold )/10000000
df_S['Sales(₹ CR)']=df_S['Sales(₹ CR)'].round(3)

In [2346]:
df_S['test8']= df_S['Specifications'].str.split('|'and'ROM').str[0]
df_S[['RAM(GB)','ROM(GB)']]= df_S['test8'].str.split('|',expand = True) 

In [2347]:
df_S['RAM(GB)']=df_S['RAM(GB)'].apply(lambda x: str(x).replace('GB','').replace('RAM',''))
df_S['RAM(GB)']=df_S['RAM(GB)'].str.replace('NA','Not Mentioned')
df_S['ROM(GB)']=df_S['ROM(GB)'].str.replace('GB','')
df_S['ROM(GB)']=df_S['ROM(GB)'].fillna("Not Mentioned")

In [2348]:
df_S['RAM(GB)']=df_S['RAM(GB)'].str.strip()
df_S['ROM(GB)']=df_S['ROM(GB)'].str.strip()

In [2349]:
def case_1(value):
  if value[-2:] == 'MB':
    return float(value[:-2])/1000
  elif value[-2:] == 'TB':
    return float(value[:-2])*1000
  else:
    return value

In [2350]:
df_S['RAM(GB)']=df_S['RAM(GB)'].map(lambda x: case_1(x))
df_S['ROM(GB)']=df_S['ROM(GB)'].map(lambda x: case_1(x))

In [2351]:
df_S['Display']= df_S['Specifications'].str.extract('\((.*?)\)')
df_S['Display']=df_S['Display'].str.replace('inch','')

In [2352]:
df_S['Screen']=df_S['Specifications'].str.extract('(Quad HD+|HD+|Full HD+ AMOLED|QGVA|AMOLED)')
df_S['Screen']=df_S['Screen'].fillna("Not Mentioned")

In [2353]:
def match(x):
    if re.search("Rear",x):
        return x 
    elif re.search("Front",x):
        return x
    else:
        return "Not Mentioned"    

In [2354]:
 df_S['test9']=df_S['Specifications'].apply(match)

In [2355]:
df_S['test9']= df_S['test9'].apply(lambda x: x[x.find("Display"):x.find("Front")])

In [2356]:
df_S['test9']=df_S['test9'].apply(lambda x:x[x.find("Display"):x.find("Rear")])

In [2357]:
df_S['test10'] = df_S['test9'].apply(lambda x:str(x).replace('Display','').replace('+',''))

In [2358]:
df_S[['Rear Camera','Front Camera']]= df_S['test10'].str.split('|',expand = True)
df_S['Rear Camera']=df_S['Rear Camera'].replace(' 0MP','')
df_S['Front Camera']=df_S['Front Camera'].replace(' 0MP','')

In [2359]:
df_S['No of Rear Camera']= df_S['Rear Camera'].str.count("MP")
df_S['No of Front Camera']= df_S['Front Camera'].str.count("MP")
df_S['No of Front Camera']= df_S['No of Front Camera'].fillna(0)

In [2360]:
df_S['Battery']= df_S['Specifications'].str.extract('(\d{4} mAh|\d{3} mAh)')
df_S['Battery']= df_S['Battery'].str.replace('mAh','')

In [2361]:
df_S['Processor']= df_S['Specifications'].str.extract('(Exynos|Mediatek|Snapdragon|Octa Core|208MHz|8 Gen 1|UMS9230|quad core|Octa Core)')
#df_S['Processor']=df_S['Processor'].replace('nan','NA')
df_S['Processor']=df_S['Processor'].fillna("Not Mentioned")

In [2362]:
# drop all test columns and unwanted columns
df_S.drop(columns=['test','test1','test2','test3','test4','test5','test6','test7','test8','test9','test10','Company Name','Ratings','Specifications','Mobile Price','offer'],inplace=True,axis=1)

In [2363]:
# value column equals to 0.0
indexNames = df_S[(df_S['Model'] == 'Metro 313') & (df_S['Ratings(*)'] =='3.8')].index
# Delete these row indexes from dataFramed
df_S.drop(indexNames , inplace=True)

In [2364]:
df_S.to_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/SamsungCleaned.csv',index=False)